In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
# %% test a specific model
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-16 12:28:00|MOD:display     |: Model Specifics:
24-05-16 12:28:00|MOD:display     |: Start Process [Data] at Thu May 16 12:28:00 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_day_ShortTest!
Callback : DynamicDataLink() , assign / unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=20,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch / event information
{'random_seed': None,
 'model_name': 'gru_day_ShortTest',
 'model_module': 

24-05-16 12:28:04|MOD:display     |: Finish Process [Data], Cost 3.7 Secs
24-05-16 12:28:04|MOD:display     |: Start Process [Fit] at Thu May 16 12:28:04 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


  0%|          | 0/10 [00:00<?, ?it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : -0.00760: 100%|██████████| 10/10 [00:01<00:00,  5.86it/s]
FirstBite Ep#  0 : loss  1.00580, train-0.00754, valid-0.00760, best-0.0076, lr1.3e-03
Valid Ep#  1 score : 0.03177: 100%|██████████| 10/10 [00:01<00:00,  7.44it/s]
FirstBite Ep#  1 : loss  0.98744, train 0.01666, valid 0.03177, best 0.0318, lr2.5e-03
Valid Ep#  2 score : 0.06209: 100%|██████████| 10/10 [00:01<00:00,  7.36it/s]
FirstBite Ep#  2 : loss  0.96170, train 0.03772, valid 0.06209, best 0.0621, lr3.8e-03
Valid Ep#  3 score : 0.07894: 100%|██████████| 10/10 [00:01<00:00,  7.39it/s]
FirstBite Ep#  3 : loss  0.93551, train 0.07118, valid 0.07894, best 0.0789, lr5.0e-03
24-05-16 12:29:08|MOD:display     |: gru_day_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.0712 Valid 0.0789 BestVal 0.0789|Cost  1.0Min, 12.3Sec/Ep
24-05-16 12:29:08|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 1.1 Min/model, 16.0 Sec/Epoch
24-05-16 12:29:08|MOD:display     |: Start Process [Test] at Thu May 16 1

                    hook_name  num_calls  total_time   avg_time
8              on_train_batch         40   45.282878   1.132072
23              on_test_batch        105   16.173702   0.154035
17           on_fit_model_end          1   10.484960  10.484960
13        on_validation_batch         40    5.669853   0.141746
4          on_fit_model_start          1    2.438516   2.438516
20        on_test_model_start          1    0.677189   0.677189
24          on_test_batch_end        105    0.138666   0.001321
14    on_validation_batch_end         40    0.050884   0.001272
9          on_train_batch_end         40    0.049904   0.001248
21   on_test_model_type_start          3    0.036904   0.012301
16           on_fit_epoch_end          4    0.036899   0.009225
27                on_test_end          1    0.018949   0.018949
15    on_validation_epoch_end          4    0.012966   0.003242
6        on_train_epoch_start          4    0.010971   0.002743
11  on_validation_epoch_start          4

In [1]:
import numpy as np
a = np.random.random((2,3))
a[a > 0.5]

array([0.68917146, 0.61358719, 0.67942325])

In [ ]:
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

In [ ]:
# prepare data
from src.interface import DataModule
DataModule.prepare_data()